In [1]:
import numpy as np
import pandas as pd
import seaborn, time

from pomegranate import BayesianNetwork

ModuleNotFoundError: No module named 'pomegranate'

In [2]:
seaborn.set_style('whitegrid')

In [3]:
"""
X = np.random.randint(2, size=(2000, 7))
X[:,3] = X[:,1]
X[:,6] = X[:,1]

X[:,0] = X[:,2]

X[:,4] = X[:,5]

model = BayesianNetwork.from_samples(X, algorithm='exact')
print(model.structure)
model.plot()
"""

"\nX = np.random.randint(2, size=(2000, 7))\nX[:,3] = X[:,1]\nX[:,6] = X[:,1]\n\nX[:,0] = X[:,2]\n\nX[:,4] = X[:,5]\n\nmodel = BayesianNetwork.from_samples(X, algorithm='exact')\nprint(model.structure)\nmodel.plot()\n"

In [4]:
def sanitize_celeb_attr_data(unsanitize_attrs_file):
    
    attr_list = []
    
    unsanitize_attrs = open(unsanitize_attrs_file,'r').readlines()
    
    for a in unsanitize_attrs:
        attr_line = a.strip('\n').split(' ')
        attrs_sanitized = [attr for attr in attr_line if attr != '']
        attr_list.append(attrs_sanitized)
        
    with open('/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt','w') as list_attr_celeb_sen:
        for attr_line in attr_list:
            list_attr_celeb_sen.write(
                ",".join(attr_line) + "\n"
            )

In [5]:
#sanitize_celeb_attr_data("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba.txt")

In [40]:
celeb_attrs = pd.read_csv("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt", 
                          sep=",", encoding='ascii')
celeb_attrs.drop('Image_ID', axis='columns', inplace=True)
celeb_attrs.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [41]:

hair_color_conditions = [
    celeb_attrs['Black_Hair'] == 1,
    celeb_attrs['Brown_Hair'] == 1,
    celeb_attrs['Blond_Hair'] == 1,
    celeb_attrs['Gray_Hair']  == 1,
    (celeb_attrs['Black_Hair'] != 1) & (celeb_attrs['Brown_Hair'] != 1) & (celeb_attrs['Blond_Hair'] != 1) & (celeb_attrs['Gray_Hair']  != 1)
]

hair_color_values = ['Black', 'Brown', 'Blond', 'Gray', 'Unknown']
celeb_attrs['Hair_Color'] = np.select(hair_color_conditions, hair_color_values)
#celeb_attrs[ ['Hair_Color','Black_Hair', 'Brown_Hair', 'Blond_Hair', 'Gray_Hair'] ]
celeb_attrs.drop(['5_o_Clock_Shadow', 'Black_Hair','Brown_Hair','Blond_Hair','Gray_Hair'], axis='columns', inplace=True)

In [42]:
hair_type_conditions = [
    celeb_attrs['Bald'] == 1,
    celeb_attrs['Bangs'] == 1,
    celeb_attrs['Straight_Hair'] == 1,
    celeb_attrs['Wavy_Hair']  == 1,
    (celeb_attrs['Bald'] != 1) & (celeb_attrs['Bangs'] != 1) & (celeb_attrs['Straight_Hair'] != 1) & (celeb_attrs['Wavy_Hair'] != 1)
]

hair_type_values = ['Bald', 'Bangs', 'Straight', 'Wavy','Unknown']

celeb_attrs['Hair_Type'] = np.select(hair_type_conditions, hair_type_values)
#celeb_attrs[ ['Hair_Type','Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'] ]
celeb_attrs.drop(['Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'], axis='columns', inplace=True)

In [44]:
celeb_attrs.head(10)

,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Big_Lips,Big_Nose,Blurry,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,...,Sideburns,Smiling,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,Hair_Color,Hair_Type
0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Brown,Straight
1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,1,Brown,Unknown
2,-1,-1,-1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Unknown,Wavy
3,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,1,-1,1,Unknown,Straight
4,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Unknown
5,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,-1,1,Brown,Wavy
6,-1,1,1,1,1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Straight
7,1,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Unknown
8,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Unknown,Bangs
9,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Wavy


In [45]:
celeb_attrs_mat = celeb_attrs.to_numpy()
print(celeb_attrs_mat.shape)

(202599, 33)


In [46]:
celeb_attrs_mat_subset = celeb_attrs_mat[:10000,:]
print(celeb_attrs_mat_subset.shape)

(10000, 33)


In [47]:
model = BayesianNetwork.from_samples(celeb_attrs_mat_subset, 
                                     state_names=celeb_attrs.columns.to_list(), 
                                     algorithm='chow-liu')
print(model.structure)

((), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), (13,), (13,), (1,), (13,), (27,))


In [49]:
#%matplotlib inline

model.plot("../reports/learned-dag-3.pdf")

In [ ]:
# 1. The labels in celeba don't capture the variation within each attribute value. Not all blond hair at the same for example. 
# So we shouldn't use just a categorical to represent that information.


In [2]:
adj_mat = [(), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), 
           (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), 
           (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), 
           (13,), (13,), (1,), (13,), (27,)]



In [31]:
parents = [3, 27]
children = [1, 11, 12, 13, 32]
get_layer_mask(parents, children, adj_mat)

parents = [1, 11, 12, 13, 32]
children = [4, 5, 9, 17, 18, 19, 20, 22, 24, 25, 28, 29, 30, 31]
get_layer_mask(parents, children, adj_mat)



array([[0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.]])

In [1]:
cd D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl


In [2]:
import torch
import numpy as np
import pandas as pd
import seaborn, time
import torch.nn.functional as F

from common import dag_utils
from common.special_modules import DAGInteractionLayer

In [3]:
adj_mat = [(), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), 
           (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), 
           (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), 
           (13,), (13,), (1,), (13,), (27,)]

In [4]:
parents = [3, 27]
children = [1, 11, 12, 13, 32]

dag_layer = DAGInteractionLayer(
    parents_list=parents, 
    children_list=children, 
    adjacency_matrix=adj_mat, 
    interm_unit_dim=2, 
    bias=True
)

In [7]:
dag_layer.diagnostic_forward(
    torch.randn(size=(4,2))
)

Input:  tensor([[ 0.3393,  0.8724],
        [ 1.4212, -0.8925],
        [-0.3542, -0.8375],
        [ 0.1024, -1.0219]])
mask_input_to_interm:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
masked_input_to_interm: tensor([[-0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000,  0.0000,  0.0000,
         -0.0000, -0.0000],
        [ 1.3936, -0.1368, -0.4320, -0.0481,  0.3139, -1.4497,  0.9508, -2.0742,
          0.7395,  0.0547]], grad_fn=<MulBackward0>)
masked_input_to_interm.Size(): torch.Size([2, 10])
X * W_1 tensor([[ 1.2157, -0.1194, -0.3768, -0.0420,  0.2738, -1.2646,  0.8294, -1.8094,
          0.6451,  0.0477],
        [-1.2437,  0.1221,  0.3855,  0.0430, -0.2801,  1.2938, -0.8486,  1.8512,
         -0.6600, -0.0488],
        [-1.1671,  0.1146,  0.3618,  0.0403, -0.2629,  1.2141, -0.7963,  1.7371,
         -0.6193, -0.0458],
        [-1.4241,  0.1398,  0.4414,  0.0492, -0.3208,  1.4814, -0.9716,  2.1196,
         -0.7557, -0.

tensor([[1.0400, 0.0000, 0.0409, 0.2570, 0.0048],
        [0.0211, 0.0878, 0.0000, 0.8600, 0.0000],
        [0.0198, 0.0824, 0.0000, 0.8070, 0.0000],
        [0.0241, 0.1006, 0.0000, 0.9847, 0.0000]], grad_fn=<ReluBackward0>)

In [28]:
layer_input = torch.randn(size=(3,2)) # (N_Batches, In_Dim)
in_features = 2
interm_unit_dim = 2
out_features = 5

mask_input_to_interm = dag_layer._get_mask_input_to_interm()
mask_interm_to_output = dag_layer._get_mask_interm_to_output()

W_input_to_interm = dag_layer.W_input_to_interm
W_interm_to_output = dag_layer.W_interm_to_output

B_input_to_interm = dag_layer.B_input_to_interm
#_input_to_interm.data.fill_(1.0)

B_interm_to_output = dag_layer.B_interm_to_output
#_interm_to_output.data.fill_(2.0)

# input to interm
print("Input: ", layer_input)
print("mask_input_to_interm: ", mask_input_to_interm)

masked_input_to_interm = W_input_to_interm.mul(mask_input_to_interm)
print("masked_input_to_interm:", masked_input_to_interm)
print("masked_input_to_interm.Size():", masked_input_to_interm.size())

interm_out = layer_input.matmul(masked_input_to_interm)
print("X * W_1", interm_out)
print("X * W_1 size()", interm_out.size())

interm_out = interm_out + B_input_to_interm

print("X * W_1 + b_1", interm_out)

interm_out = F.relu(interm_out)
print("U = ReLU(X * W_1 + b_1)", interm_out)
print("U.size()", interm_out.size())


Input:  tensor([[ 0.3870, -0.2251],
        [-0.4509,  0.5795],
        [-0.9146, -0.6917]])
mask_input_to_interm:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
masked_input_to_interm: tensor([[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000,
          0.0000, -0.0000],
        [ 0.7641, -0.9518, -0.3071, -1.4622,  1.1528, -0.8837,  1.2768,  1.0276,
         -0.6804,  2.2462]], grad_fn=<MulBackward0>)
masked_input_to_interm.Size(): torch.Size([2, 10])
W_1.t * X tensor([[-0.1720,  0.2142,  0.0691,  0.3291, -0.2595,  0.1989, -0.2874, -0.2313,
          0.1531, -0.5055],
        [ 0.4428, -0.5515, -0.1780, -0.8473,  0.6681, -0.5121,  0.7399,  0.5955,
         -0.3943,  1.3016],
        [-0.5285,  0.6584,  0.2124,  1.0114, -0.7974,  0.6112, -0.8832, -0.7108,
          0.4706, -1.5537]], grad_fn=<MmBackward>)
W_1.t * X size() torch.Size([3, 10])
W_1.t * X + b_1 tensor([[ 0.8280,  1.2142,  1.0691,  1.3291,  0.7405

In [29]:
W_interm_to_output.size()

torch.Size([10, 5])

In [31]:

# interm to output
print("mask_interm_to_output ", mask_interm_to_output)

masked_interm_to_output = W_interm_to_output.mul(mask_interm_to_output)
print("masked_interm_to_output: ", masked_interm_to_output)

out = interm_out.matmul(masked_interm_to_output)
print("U * W_2", out)
print("U * W_2 size", out.size())

out = out + B_interm_to_output
print("U * W_2 + b_2", out)

out = F.relu(out)
print("Out = ReLU(U * W_2 + b_2)", out)
print("Out.size()", out.size())

mask_interm_to_output  tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]])
masked_interm_to_output:  tensor([[-0.0941,  0.0000, -0.0000,  0.0000,  0.0000],
        [-0.4259, -0.0000, -0.0000,  0.0000, -0.0000],
        [ 0.0000, -0.5561,  0.0000, -0.0000,  0.0000],
        [ 0.0000, -0.0805,  0.0000, -0.0000,  0.0000],
        [ 0.0000, -0.0000,  0.3383, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.5644,  0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0000, -0.1357, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.9244,  0.0000],
        [ 0.0000,  0.0000, -0.0000,  0.0000,  0.1927],
        [-0.0000,  0.0000, -0.0000, -0.0000,  0.0635]], grad_fn=<MulBackward0>)
W_2.t * U tensor([[-0.5951, -0.7015, -0.4261, -0.8073,  0.2537],
        [-

In [8]:
print(dag_utils.get_dag_layers(adj_mat))

print(dag_utils.get_layer_mask(parents, children, 2, adj_mat))
parents=[2, 6, 7, 10, 14, 15, 21, 23, 26]
children=[8, 16]
print(dag_utils.get_layer_mask(parents, children, 3, adj_mat))

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
